In [1]:
import glob
import pandas as pd
import os
import re # For filtering sheet names

In [2]:
# --- 1. Define the path to your data ---
# This path comes from our project structure
RAW_DATA_PATH = "../data/raw/"

# --- 2. Get all Excel files ---
all_excel_files = glob.glob(RAW_DATA_PATH + "Padang Nyiru*.xlsx")
print(f"Found {len(all_excel_files)} files:")
print(all_excel_files)

Found 12 files:
['../data/raw\\Padang Nyiru April 2024.xlsx', '../data/raw\\Padang Nyiru August 2024.xlsx', '../data/raw\\Padang Nyiru December 2024.xlsx', '../data/raw\\Padang Nyiru February 2024.xlsx', '../data/raw\\Padang Nyiru January 2024.xlsx', '../data/raw\\Padang Nyiru July 2024.xlsx', '../data/raw\\Padang Nyiru June 2024.xlsx', '../data/raw\\Padang Nyiru March 2024.xlsx', '../data/raw\\Padang Nyiru May 2024.xlsx', '../data/raw\\Padang Nyiru November 2024.xlsx', '../data/raw\\Padang Nyiru October 2024.xlsx', '../data/raw\\Padang Nyiru September 2024.xlsx']


In [3]:
# --- 3. Reset lists AND define expected structures ---
standard_dfs_list = [] # Reset list
june_dfs_list = []     # Reset list

# Define column names for the June format
june_col_names = ['Date_Raw', 'Time_Range', 'Motorcycle', 'Car,MPVs', 'Truck/Lorry', 'Bus']

# Define the CORE columns we EXPECT in a standard (non-June) sheet
core_standard_cols = {
    'From', 'To',
    'Phase1(Dari Jalan Temenggung)_Motorcycle', 'Phase1(Dari Jalan Temenggung)_Car',
    'Phase1(Dari Jalan Temenggung)_Truck/Lorry', 'Phase1(Dari Jalan Temenggung)_Bus',
    'Phase2(Dari Jalan Bendahara)_Motorcycle', 'Phase2(Dari Jalan Bendahara)_Car',
    'Phase2(Dari Jalan Bendahara)_Truck/Lorry', 'Phase2(Dari Jalan Bendahara)_Bus'
}
# Define the exact list for selection later (maintains order)
final_standard_cols_list = [
    'From', 'To',
    'Phase1(Dari Jalan Temenggung)_Motorcycle', 'Phase1(Dari Jalan Temenggung)_Car',
    'Phase1(Dari Jalan Temenggung)_Truck/Lorry', 'Phase1(Dari Jalan Temenggung)_Bus',
    'Phase2(Dari Jalan Bendahara)_Motorcycle', 'Phase2(Dari Jalan Bendahara)_Car',
    'Phase2(Dari Jalan Bendahara)_Truck/Lorry', 'Phase2(Dari Jalan Bendahara)_Bus'
]

In [4]:
# --- 4. Loop and Load ---
debug_printed = False

for file_path in all_excel_files:
    print(f"\nProcessing file: {file_path}")

    try:
        file_name = os.path.basename(file_path)
        month = file_name.split(' ')[2]
    except Exception as e:
        month = "Unknown"

    # --- Get all sheet names ---
    try:
        xl = pd.ExcelFile(file_path)
        sheet_names = xl.sheet_names
    except Exception as e:
        print(f"  Could not read file {file_name}: {e}")
        continue

    # --- Find data sheets (any sheet starting with a number) ---
    data_sheet_names = [s for s in sheet_names if re.search(r"^\d{1,2}", s.strip())]

    if not data_sheet_names:
        print(f"  No valid data sheets found in {file_name}.")
        continue

    print(f"  Found {len(data_sheet_names)} data sheets. Loading...")

    # --- MAIN LOGIC: Check for June vs. Other ---
    if "June" in file_name:
            # --- JUNE FORMAT LOGIC (Revised ffill placement) ---
            june_p1_cols = ['Date_Raw', 'Time_Range', 'Motorcycle', 'Car,MPVs', 'Truck/Lorry', 'Bus']
            june_p2_cols_read = ['Date_Raw', 'Time_Range', 'Motorcycle', 'Car,MPVs', 'Truck/Lorry', 'Bus']
    
            for sheet_name in data_sheet_names:
                try:
                    # Load Phase 1 table
                    df_p1 = pd.read_excel(
                        file_path, sheet_name=sheet_name, header=None,
                        skiprows=4, nrows=24, usecols="A:F"
                    )
                    df_p1.columns = june_p1_cols
                    # --- Fill Date_Raw for Phase 1 ---
                    df_p1['Date_Raw'] = df_p1['Date_Raw'].ffill()
                    df_p1['Phase'] = 'Phase1(Dari Jalan Temenggung)'
    
                    # Load Phase 2 table
                    df_p2 = pd.read_excel(
                        file_path, sheet_name=sheet_name, header=None,
                        skiprows=4, nrows=24, usecols="I:N" # Corrected columns
                    )
                    df_p2.columns = june_p2_cols_read
                    # --- Fill Date_Raw for Phase 2 ---
                    df_p2['Date_Raw'] = df_p2['Date_Raw'].ffill()
                    df_p2['Phase'] = 'Phase2(Dari Jalan Bendahara)'
    
                    # Combine P1 and P2
                    df_sheet_combined = pd.concat([df_p1, df_p2], ignore_index=True)
                    df_sheet_combined['Month'] = 'June'
                    df_sheet_combined['Sheet_Name'] = sheet_name
                    # Date_Raw is already filled now
    
                    june_dfs_list.append(df_sheet_combined)
    
                except Exception as e:
                    print(f"    Failed to load June sheet '{sheet_name}': {e}")
    else:
        # --- STANDARD FORMAT LOGIC (All OTHER 11 files, including January) ---
        for sheet_name in data_sheet_names:
            try:
                 # *** Use header=[2, 3] for ALL standard files ***
                df = pd.read_excel(file_path, sheet_name=sheet_name, header=[2, 3], nrows=24)

                # Flatten headers
                new_cols = []
                for col in df.columns:
                    part1 = " ".join(str(col[0]).split()).strip()
                    part2 = " ".join(str(col[1]).split()).strip()
                    if part2 in ('From', 'To'): new_cols.append(part2)
                    else: new_cols.append(f"{part1}_{part2}")
                df.columns = new_cols

                # --- Validation ---
                generated_cols = set(df.columns)
                if not core_standard_cols.issubset(generated_cols):
                    print(f"    WARNING: Sheet '{sheet_name}' in {file_name} missing core data columns. Skipping.")
                    if not debug_printed: # Print debug info once if validation fails
                         print("\n--- DEBUG (Standard): EXPECTED CORE ---"); print(sorted(list(core_standard_cols)))
                         print("\n--- DEBUG (Standard): ACTUAL GENERATED ---"); print(sorted(list(generated_cols)))
                         debug_printed = True
                    continue

                # Select only the needed columns
                try:
                    df = df[final_standard_cols_list]
                except KeyError as e:
                     print(f"    ERROR: Sheet '{sheet_name}' passed core check but failed final selection (KeyError: {e}). Skipping.")
                     continue

                # Add metadata
                df['Date'] = sheet_name
                df['Month'] = month
                standard_dfs_list.append(df) # Append valid data to the STANDARD list

            except Exception as e: print(f"    Failed to load standard sheet '{sheet_name}': {e}")


Processing file: ../data/raw\Padang Nyiru April 2024.xlsx
  Found 30 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru August 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru December 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru February 2024.xlsx
  Found 29 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru January 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru July 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru June 2024.xlsx
  Found 30 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru March 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru May 2024.xlsx
  Found 31 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru November 2024.xlsx
  Found 30 data sheets. Loading...

Processing file: ../data/raw\Padang Nyiru October 2

In [8]:
display(standard_dfs_list[0].head(50))

,From,To,Phase1(Dari Jalan Temenggung)_Motorcycle,Phase1(Dari Jalan Temenggung)_Car,Phase1(Dari Jalan Temenggung)_Truck/Lorry,Phase1(Dari Jalan Temenggung)_Bus,Phase2(Dari Jalan Bendahara)_Motorcycle,Phase2(Dari Jalan Bendahara)_Car,Phase2(Dari Jalan Bendahara)_Truck/Lorry,Phase2(Dari Jalan Bendahara)_Bus,Date,Month
0,00:00:00,01:00:00,54,231,2,0,53,234,1,0,1 April,April
1,01:00:00,02:00:00,51,182,1,0,58,176,2,0,1 April,April
2,02:00:00,03:00:00,39,121,0,0,33,121,0,0,1 April,April
3,03:00:00,04:00:00,42,97,0,0,23,97,0,0,1 April,April
4,04:00:00,05:00:00,31,43,0,0,21,61,0,0,1 April,April
5,05:00:00,06:00:00,25,198,0,0,16,198,1,0,1 April,April
6,06:00:00,07:00:00,61,257,2,1,61,257,1,0,1 April,April
7,07:00:00,08:00:00,108,393,6,0,188,465,7,0,1 April,April
8,08:00:00,09:00:00,95,318,5,0,174,434,6,0,1 April,April
9,09:00:00,10:00:00,99,344,9,0,168,419,7,0,1 April,April


In [9]:
display(june_dfs_list[0
        ].tail(100))

,Date_Raw,Time_Range,Motorcycle,"Car,MPVs",Truck/Lorry,Bus,Phase,Month,Sheet_Name
0,2024-06-01,00:00 - 01:00,63,387,8,0,Phase1(Dari Jalan Temenggung),June,6.1.24
1,2024-06-01,01:00 - 02:00,24,184,4,0,Phase1(Dari Jalan Temenggung),June,6.1.24
2,2024-06-01,02:00 - 03:00,63,386,6,0,Phase1(Dari Jalan Temenggung),June,6.1.24
3,2024-06-01,03:00 - 04:00,19,169,3,0,Phase1(Dari Jalan Temenggung),June,6.1.24
4,2024-06-01,04:00 - 05:00,34,234,4,0,Phase1(Dari Jalan Temenggung),June,6.1.24
5,2024-06-01,05:00 - 06:00,27,210,3,0,Phase1(Dari Jalan Temenggung),June,6.1.24
6,2024-06-01,06:00 - 07:00,24,183,4,0,Phase1(Dari Jalan Temenggung),June,6.1.24
7,2024-06-01,07:00 - 08:00,69,414,9,0,Phase1(Dari Jalan Temenggung),June,6.1.24
8,2024-06-01,08:00 - 09:00,34,242,4,0,Phase1(Dari Jalan Temenggung),June,6.1.24
9,2024-06-01,09:00 - 10:00,24,193,3,0,Phase1(Dari Jalan Temenggung),June,6.1.24


In [11]:
# --- 5. Combine into TWO separate DataFrames ---
if standard_dfs_list:
    df_standard_raw = pd.concat(standard_dfs_list, ignore_index=True)
    print(f"\nSuccessfully loaded {len(df_standard_raw)} rows from {len(standard_dfs_list)} STANDARD sheets.") # Changed print logic
else:
    print("\nNo standard data was loaded.")
    df_standard_raw = pd.DataFrame() # Create empty df if none loaded

if june_dfs_list:
    df_june_raw = pd.concat(june_dfs_list, ignore_index=True)
    print(f"\nSuccessfully loaded {len(df_june_raw)} rows from {len(june_dfs_list)} JUNE sheets.") # Changed print logic
else:
    print("\nNo June data was loaded.")
    df_june_raw = pd.DataFrame() # Create empty df if none loaded


Successfully loaded 8064 rows from 336 STANDARD sheets.

Successfully loaded 1440 rows from 30 JUNE sheets.


In [12]:
display(df_june_raw.tail(10), df_standard_raw)

,Date_Raw,Time_Range,Motorcycle,"Car,MPVs",Truck/Lorry,Bus,Phase,Month,Sheet_Name
1430,2024-06-30,14:00 - 15:00,44,895,3,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1431,2024-06-30,15:00 - 16:00,56,855,3,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1432,2024-06-30,16:00 - 17:00,44,900,3,2,Phase2(Dari Jalan Bendahara),June,6.30.24
1433,2024-06-30,17:00 - 18:00,69,702,3,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1434,2024-06-30,18:00 - 19:00,47,747,5,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1435,2024-06-30,19:00 - 20:00,57,264,2,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1436,2024-06-30,20:00 - 21:00,63,350,5,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1437,2024-06-30,21:00 - 22:00,54,583,3,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1438,2024-06-30,22:00 - 23:00,60,793,3,1,Phase2(Dari Jalan Bendahara),June,6.30.24
1439,2024-06-30,23:00 - 00:00,70,675,4,1,Phase2(Dari Jalan Bendahara),June,6.30.24


,From,To,Phase1(Dari Jalan Temenggung)_Motorcycle,Phase1(Dari Jalan Temenggung)_Car,Phase1(Dari Jalan Temenggung)_Truck/Lorry,Phase1(Dari Jalan Temenggung)_Bus,Phase2(Dari Jalan Bendahara)_Motorcycle,Phase2(Dari Jalan Bendahara)_Car,Phase2(Dari Jalan Bendahara)_Truck/Lorry,Phase2(Dari Jalan Bendahara)_Bus,Date,Month
0,00:00:00,01:00:00,54.0,231,2.0,0.0,53.0,234,1,0,1 April,April
1,01:00:00,02:00:00,51.0,182,1.0,0.0,58.0,176,2,0,1 April,April
2,02:00:00,03:00:00,39.0,121,0.0,0.0,33.0,121,0,0,1 April,April
3,03:00:00,04:00:00,42.0,97,0.0,0.0,23.0,97,0,0,1 April,April
4,04:00:00,05:00:00,31.0,43,0.0,0.0,21.0,61,0,0,1 April,April
...,...,...,...,...,...,...,...,...,...,...,...,...
8059,19:00:00,20:00:00,163.0,442,8.0,4.0,182.0,866,14,1,30 September,September
8060,20:00:00,21:00:00,130.0,334,6.0,4.0,152.0,691,12,1,30 September,September
8061,21:00:00,22:00:00,150.0,398,8.0,4.0,140.0,643,14,1,30 September,September
8062,22:00:00,23:00:00,79.0,172,6.0,5.0,127.0,559,10,1,30 September,September


In [13]:
# --- 6. Harmonize Standard Data ---
df_standard_tidy = pd.DataFrame() # Create empty df
if not df_standard_raw.empty:
    print("--- Harmonizing Standard Data ---")

    id_vars = ['Date', 'Month', 'From', 'To']
    value_vars = [col for col in df_standard_raw.columns if 'Phase' in col]
    df_standard_tidy = df_standard_raw.melt(
        id_vars=id_vars, value_vars=value_vars,
        var_name='Phase_Vehicle', value_name='Count'
    )
    df_standard_tidy['Day'] = df_standard_tidy['Date'].str.extract(r"(\d{1,2})")[0]
    df_standard_tidy['Full_Date_Str'] = df_standard_tidy['Day'] + ' ' + df_standard_tidy['Month'] + ' 2024'
    df_standard_tidy['Full_Date'] = pd.to_datetime(df_standard_tidy['Full_Date_Str'], errors='coerce', format='%d %B %Y')
    splits = df_standard_tidy['Phase_Vehicle'].str.split('_', expand=True)
    df_standard_tidy['Phase'] = splits[0]
    df_standard_tidy['Vehicle_Type'] = splits[1]
    df_standard_tidy = df_standard_tidy.rename(columns={'From': 'Time_From', 'To': 'Time_To'})
    df_standard_tidy['Time_From'] = df_standard_tidy['Time_From'].astype(str)
    df_standard_tidy['Time_To'] = df_standard_tidy['Time_To'].astype(str)

    final_cols = ['Full_Date', 'Time_From', 'Time_To', 'Phase', 'Vehicle_Type', 'Count']
    df_standard_tidy = df_standard_tidy[final_cols].dropna(subset=['Full_Date'])
    print("Standard data harmonized.")

--- Harmonizing Standard Data ---
Standard data harmonized.


In [14]:
# --- 7. Harmonize June Data ---
df_june_tidy = pd.DataFrame() # Create empty df
if not df_june_raw.empty:
    print("\n--- Harmonizing June Data ---")

    id_vars = ['Date_Raw', 'Time_Range', 'Phase', 'Month', 'Sheet_Name']
    value_vars = ['Motorcycle', 'Car,MPVs', 'Truck/Lorry', 'Bus']
    df_june_tidy = df_june_raw.melt(
        id_vars=id_vars, value_vars=value_vars,
        var_name='Vehicle_Type', value_name='Count'
    )
    df_june_tidy['Vehicle_Type'] = df_june_tidy['Vehicle_Type'].replace('Car,MPVs', 'Car')
    df_june_tidy['Full_Date'] = pd.to_datetime(df_june_tidy['Date_Raw']).dt.normalize()
    time_splits = df_june_tidy['Time_Range'].str.split(' - ', expand=True)
    df_june_tidy['Time_From'] = time_splits[0]
    df_june_tidy['Time_To'] = time_splits[1]

    final_cols = ['Full_Date', 'Time_From', 'Time_To', 'Phase', 'Vehicle_Type', 'Count']
    df_june_tidy = df_june_tidy[final_cols]
    print("June data harmonized.")


--- Harmonizing June Data ---
June data harmonized.


In [15]:
# --- 8. Combine into one Tidy DataFrame ---
df_final = pd.DataFrame() # Create an empty df
# Check if tidy dataframes exist and are not empty before concatenating
df_list_to_concat = []
if not df_standard_tidy.empty:
    df_list_to_concat.append(df_standard_tidy)
    print("\n--- Standard tidy data found ---")
if not df_june_tidy.empty:
    df_list_to_concat.append(df_june_tidy)
    print("\n--- June tidy data found ---")

if df_list_to_concat:
    df_final = pd.concat(df_list_to_concat, ignore_index=True)
    print("\n--- All tidy data combined! ---")
    print(f"Total rows in combined tidy data: {len(df_final)}")
else:
    print("\n--- No valid tidy dataframes to combine ---")


--- Standard tidy data found ---

--- June tidy data found ---

--- All tidy data combined! ---
Total rows in combined tidy data: 70272


In [16]:
display(df_final.tail(10))

,Full_Date,Time_From,Time_To,Phase,Vehicle_Type,Count
70262,2024-06-30,14:00,15:00,Phase2(Dari Jalan Bendahara),Bus,1
70263,2024-06-30,15:00,16:00,Phase2(Dari Jalan Bendahara),Bus,1
70264,2024-06-30,16:00,17:00,Phase2(Dari Jalan Bendahara),Bus,2
70265,2024-06-30,17:00,18:00,Phase2(Dari Jalan Bendahara),Bus,1
70266,2024-06-30,18:00,19:00,Phase2(Dari Jalan Bendahara),Bus,1
70267,2024-06-30,19:00,20:00,Phase2(Dari Jalan Bendahara),Bus,1
70268,2024-06-30,20:00,21:00,Phase2(Dari Jalan Bendahara),Bus,1
70269,2024-06-30,21:00,22:00,Phase2(Dari Jalan Bendahara),Bus,1
70270,2024-06-30,22:00,23:00,Phase2(Dari Jalan Bendahara),Bus,1
70271,2024-06-30,23:00,00:00,Phase2(Dari Jalan Bendahara),Bus,1


In [17]:
# --- 9. Aggregate and Pivot to Final Format ---
if not df_final.empty:
    print("\n--- Aggregating and Pivoting to Final Format ---")

    # Clean the 'Count' column first
    print("Cleaning 'Count' column before aggregation...")
    df_final['Count'] = pd.to_numeric(df_final['Count'], errors='coerce').fillna(0).astype(int)
    print(f"'Count' column cleaned. Dtype: {df_final['Count'].dtype}")

    # Create a single 'Datetime' column to group by
    df_final['Time_From'] = df_final['Time_From'].astype(str).str.extract(r'(\d{2}:\d{2})')[0]
    df_final.dropna(subset=['Time_From'], inplace=True) # Drop rows with invalid time

    try:
        df_final['Datetime'] = pd.to_datetime(df_final['Full_Date'].astype(str) + ' ' + df_final['Time_From'])
    except Exception as e:
        print(f"Error creating Datetime column: {e}")
        # Add debugging for time format issues
        print("Unique Time_From values causing issues:")
        print(df_final[pd.to_datetime(df_final['Full_Date'].astype(str) + ' ' + df_final['Time_From'], errors='coerce').isna()]['Time_From'].unique())
        raise e

    # Group by Datetime and Vehicle Type, then sum the counts
    df_aggregated = df_final.groupby(['Datetime', 'Vehicle_Type'])['Count'].sum().reset_index()

    # Pivot the table
    df_pivoted = df_aggregated.pivot(
        index='Datetime', columns='Vehicle_Type', values='Count'
    ).reset_index().fillna(0)

    # Clean up column names
    df_pivoted.columns.name = None
    df_pivoted = df_pivoted.rename(columns={'Truck/Lorry': 'Truck_Lorry'})

    # Ensure all expected vehicle columns exist
    expected_vehicle_cols = ['Motorcycle', 'Car', 'Truck_Lorry', 'Bus']
    for col in expected_vehicle_cols:
        if col not in df_pivoted.columns:
            df_pivoted[col] = 0

    # Select and reorder columns
    final_cols_agg = ['Datetime'] + expected_vehicle_cols
    df_final_cleaned = df_pivoted[final_cols_agg]

    print("Data successfully aggregated and pivoted.")
    display(df_final_cleaned.head())

    # --- 10. Save the final cleaned data ---
    CLEAN_DATA_PATH = "../data/processed/cleaned_traffic_data_aggregated.csv"
    os.makedirs(os.path.dirname(CLEAN_DATA_PATH), exist_ok=True)

    df_final_cleaned.to_csv(CLEAN_DATA_PATH, index=False)
    print(f"\nFinal aggregated data saved to: {CLEAN_DATA_PATH}")

else:
    print("\n'df_final' not found or empty. Cannot aggregate.")


--- Aggregating and Pivoting to Final Format ---
Cleaning 'Count' column before aggregation...
'Count' column cleaned. Dtype: int64
Data successfully aggregated and pivoted.


,Datetime,Motorcycle,Car,Truck_Lorry,Bus
0,2024-01-01 00:00:00,77,291,0,0
1,2024-01-01 01:00:00,102,291,0,0
2,2024-01-01 02:00:00,161,383,0,0
3,2024-01-01 03:00:00,90,287,0,0
4,2024-01-01 04:00:00,113,301,1,0



Final aggregated data saved to: ../data/processed/cleaned_traffic_data_aggregated.csv


In [18]:
display(df_final_cleaned.tail())

,Datetime,Motorcycle,Car,Truck_Lorry,Bus
8776,2024-12-31 19:00:00,408,1674,18,4
8777,2024-12-31 20:00:00,362,1738,17,3
8778,2024-12-31 21:00:00,306,1377,13,3
8779,2024-12-31 22:00:00,348,1533,16,3
8780,2024-12-31 23:00:00,326,1410,16,3
